#  Import Package

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import tqdm
import random
import datetime
from sklearn.metrics import mean_squared_log_error
from datetime import datetime
import calendar
import itertools
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
#import seaborn as sns
#import statsmodels.api as sm
#from statsmodels.tsa.api import ExponentialSmoothing
#from statsmodels.tsa.stattools import adfuller
from pandas import read_csv
from numpy import nan
from numpy import isnan
from numpy import split
from numpy import array
from matplotlib import pyplot
import xlrd
from collections import defaultdict
from datetime import date,datetime
import math
from sklearn.cluster import KMeans
from sklearn import datasets
import copy
from scipy import stats
import time
from pylab import *
plt.rcParams['font.sans-serif'] = ['SimHei']
import argparse
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder
import os
from math import sqrt
from numpy import concatenate
from pandas import DataFrame
from pandas import concat
#导入scipy模块
import joblib
#%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib.pylab import style #自定义图表风格
style.use('ggplot')
# 解决中文乱码问题
plt.rcParams['font.sans-serif'] = ['Simhei']
# 解决坐标轴刻度负号乱码
plt.rcParams['axes.unicode_minus'] = False


Bad key "nbagg.transparent" on line 426 in
D:\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_path" on line 509 in
D:\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_args" on line 512 in
D:\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution
In D:\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.

In [2]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA

# Load Data

In [3]:
abc=30#表示30min为一段
#df0=pd.read_csv(r'F:\博士科研资料\博士开题及其素材\内容1：新零售预测文献以及相关数据集\数据6--sales_detail.csv\tyzx3month.csv',encoding ='GB2312')
gdf0=pd.read_csv(open('F:\博士科研资料\博士开题及其素材\加油站数据\export11.csv'))#,encoding='gb18030')
gdf0['Time1'] = pd.to_datetime(gdf0['Time'],errors='coerce') 
gdf0['Hour'] = gdf0['Time1'].dt.hour.fillna(0).astype("int")    #转化提取小时
gdf0['Minute'] = gdf0['Time1'].dt.minute.fillna(0).astype("int") #转化提取分钟
gdf0['Hour']=gdf0['Hour'].astype(str)
gdf0['Date'] = pd.to_datetime(gdf0['Date'])
gdf0['T'] = gdf0['Hour'].apply(lambda x : int(x))+gdf0['Minute'].apply (lambda x :round(x/60,2))
gdf0['volume'] = gdf0['volume']
gdf0['MinuteLabel']=gdf0['Hour'].apply(lambda x : math.floor((int(x)*60)/abc))+gdf0['Minute'].apply (lambda x :math.floor(x/abc))#表示该时刻点属于哪个区间,因为有0，所以向下取整数
#gdf0=gdf0.sort_values(by=['Date','Time'],ascending=[True,True]) #df3是没有的，用0不足的dataframe
day_name00=gdf0.groupby(["NAME"])         #
for name,group in day_name00:#group是星期数一样的类目
    if name=='92H':
        df0=group
df0=df0[df0['Date'] >= "2020-06-01"] 
print('00df0.shape[0]',df0.shape[0])
df0= df0[~( df0['Date'].isin(["2020-06-11","2020-08-11", "2020-08-25", "2020-09-11"]) )]###删除这些行，因为很多0
print('11df0.shape[0]',df0.shape[0])

00df0.shape[0] 209937
11df0.shape[0] 207441


# Data Preprocessing

In [4]:
df2 = df0.groupby(["Date","MinuteLabel"])['volume'].sum().reset_index(name ='UnitSales')#只留下小时销量数据
df2['Date'] = pd.to_datetime(df2['Date']) 
df2['MinuteLabel']=df2['MinuteLabel'].astype(str)
#dfghu=df2.sort_values(by=['UnitSales'],ascending=[True]) 
#print('dfghu.loc[0,UnitSales]',dfghu.iloc[0,2])#最小值
#print('dfghu.loc[0,UnitSales]',dfghu.iloc[dfghu.shape[0]-1,2])#最大值
#df2['UnitSales']=df2['UnitSales'].apply(lambda x: (x-dfghu.iloc[0,2])/(dfghu.iloc[dfghu.shape[0]-1,2]-dfghu.iloc[0,2]))
df2['MinuteLabel']=df2['MinuteLabel'].astype(int)#
idx=list(range(int(24*60/abc)))                                                         
###3###3####用0铺满
group1=df2.groupby(["Date"])
j=0
for name,group2 in group1:#group2是星期数、日期数一样的类目，即一天  
     group2=group2.sort_values(by=['MinuteLabel'],ascending=True,na_position='first')  # 按时间排序                  
     group2 = group2.set_index('MinuteLabel')#group2设置索引为Hour         
     group2 = group2.reindex(idx, fill_value=0)
     group2['MinuteLabel'] = group2.index          
     group2['Date']=name
     if j==0:
        df3=group2
     else:      
        df3=pd.concat([df3,group2], axis=0, ignore_index=True)#axis:合并方式，默认0表示按列合并，1表示按行合并;ignore_index:是否忽略索引                   
     j=j+1
df3=df3.sort_values(by=['Date','MinuteLabel'],ascending=[True,True]) #df3是没有的，用0不足的dataframe
display(df3.head(3))#df3是没有的全部填充为0的
df = pd.DataFrame(columns=['timestamp', 'value'])###初始化
df3['AllMinutes']=df3['MinuteLabel'].apply(lambda x: time.strftime("%H:%M:%S", time.gmtime( float((x+1)*abc*60))))
for i in range(df3.shape[0]):
    if df3.loc[i,'AllMinutes']=="00:00:00": 
        df3.loc[i,'AllMinutes']="23:59:00"  ###1月20日0:00:00表示的是1月20日24:00:00,而无法取24h，所以设为23.59
#将秒转换为时分秒的形式
df['timestamp']=df3['Date'].astype('str')+ ' ' + df3['AllMinutes'].astype('str')
df['timestamp'] = pd.to_datetime(df['timestamp'])#为什么成了选取偶数
#df['value']=df5['UnitSales']+100#将小时销量转换为value值，且为了0不可用，所有的值增加100.
df['value']=df3['UnitSales']
df=df.sort_values(by=['timestamp'],ascending=[True]) 
df['Date']=df['timestamp'].apply(lambda x: x.strftime("%Y-%m-%d"))#提取时间戳中的日期
### READ DATA AND CREATE FEATURES FOR year, month, day, hour ###
df['MinuteLabel']=df3['MinuteLabel']
df=df.sort_values(by=['timestamp','MinuteLabel'],ascending=[True,True]) #df3是没有的，用0不足的dataframe
##df=df.iloc[152*(int(24*60/abc)):,:]
d1_sale=pd.DataFrame()
d1_sale=df

,Date,UnitSales,MinuteLabel
0,2020-06-01,18.46,0
1,2020-06-01,105.28,1
2,2020-06-01,0.00,2


In [5]:
#DFStockOut=df0.iloc[:,[1,2,3,7,14,15]]
DFStockOut=df0.iloc[:,[6,7,11,3,10,12]]
c=3
a=DFStockOut.shape[1]
ReStartStock=13790#10000#该变量表示补货到达后的初始库存
DFStockOut['OurLeaveStock']=DFStockOut['Date'].apply(lambda x : ReStartStock)#6
DFStockOut['OurBuHuoStart']=DFStockOut['Date'].apply(lambda x : 0)#补货触发时刻7
b=a+1
DFStockOut['Date']=pd.to_datetime(DFStockOut['Date'])
#from datetime import timedelta
#display(DFStockOut.head(2))

StockOutTime=[]
DHCS1=0#表示订货次数
#display(DFStockOut.head(5))

DFStockOut['AllMinutes']=DFStockOut['MinuteLabel'].apply(lambda x: time.strftime("%H:%M:%S", time.gmtime( float((x+1)*abc*60))))
#将秒转换为时分秒的形式

for i in range(DFStockOut.shape[0]):
    if DFStockOut.iloc[i,-1]=="00:00:00":
        #print('i is',i)
        #print(DFStockOut.iloc[i,-1])   
        DFStockOut.iloc[i,-1]="23:59:00"  ###1月20日0:00:00表示的是1月20日24:00:00,而无法取24h，所以设为23.59
    
DFStockOut['timestamp']=DFStockOut['Date'].astype('str')+ ' ' + DFStockOut['AllMinutes'].astype('str')

DFStockOut['timestamp'] = pd.to_datetime(DFStockOut['timestamp'])#pd.to_datetime它无法转换

print(DFStockOut.shape)#(321121, 9)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

(207441, 10)


D:\anaconda\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# SVM multi-step-ahead forecasting

## Define functions 

In [6]:
#分成输入一天，输出一天
def to_supervised2(data, n_input, n_out,step):#数据裂变划分
    #将所有数据转换为输入(X[..,len(x_input),1])、y[..,len(data[in_end:out_end, 0])]
    X, y = list(), list()
    in_start = 0
    for _ in range(len(data)):# 逐步遍历整个历史数据
        in_end = in_start + n_input## 定义输入序列的结束点
        out_end = in_end + n_out
        if out_end < (len(data)+1):# 确保有足够的数据
            x_input = data[in_start:in_end, 0]
            #x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])
        #in_start += step    # 依次推进时间
        in_start += 1    # 依次推进时间
    return array(X), array(y)

###00前期数据准备
def DataPreprogress(df,step):
    label1 = df.value[:].values #- init1#没必要取对数吧
    label1=np.array(label1).reshape(len(label1),1)#1维变2维
    Xx, Yy = to_supervised2(label1, n_input, n_out,step)#准备数据#00
    return Xx, Yy
multi_model ='MultiSVR'#'ChainSVR'# 'ChainSVR'
if multi_model == 'RandomForest':
    # Random Forest Regression (supports multi-output natively)
    forest = RandomForestRegressor(criterion='mse', bootstrap=True)
    # Creating a pipeline
    pipe = Pipeline(steps=[('preprocess', 'passthrough'), 
                           ('forest', forest)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'preprocess': [None, StandardScaler()], 
                   'forest__n_estimators': stats.randint(low=200, high=1000),
                   'forest__max_depth': [1, 3, 5, None], 
                   'forest__max_samples': stats.uniform(loc=0.2, scale=0.8),
                  }
elif multi_model == 'ChainSVR':
    # Support Vector Regression (does NOT support multi-output natively)
    svr = RegressorChain(SVR(kernel='rbf', gamma='scale'))
    # Creating a pipeline
    pipe = Pipeline(steps=[('preprocess', 'passthrough'), 
                           ('svr', svr)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'preprocess': [None, StandardScaler()], 
                   'svr__base_estimator__C': stats.lognorm(1e0, 1e3),
                   'svr__base_estimator__epsilon': stats.lognorm(1e-5, 1e-2),
                  }
elif multi_model == 'DecisionTree':
    # Decision tree regressor 
    tree = DecisionTreeRegressor()
    # Creating a pipeline
    pipe = Pipeline(steps=[('tree', tree)])
    param_dists = {'tree__criterion': ['mse', 'mae'],
                   'tree__max_depth': [2, 4, 6, 8, None],
                   'tree__min_samples_leaf': stats.randint(low=1, high=9)
                   }
          
elif multi_model == 'MultiSVR':
    # Support Vector Regression (does NOT support multi-output natively)
    svr = MultiOutputRegressor(SVR(kernel='rbf', gamma='scale'))
    # Creating a pipeline
    pipe = Pipeline(steps=[('preprocess', 'passthrough'), ('svr', svr)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'preprocess': [None, StandardScaler()], 
                   'svr__estimator__C': stats.lognorm(1e0, 1e3),
                   'svr__estimator__epsilon': stats.lognorm(1e-5, 1e-2),
                  }
elif multi_model == 'PCA+SVR':
    # Principal Component Analysis (PCA) is used for decomposing 
    # (i.e. projecting) features into the lower-dimensional space
    # while retaining maximum amount of the variance.
    pca = PCA(whiten=True, svd_solver='full')
    # Support Vector Regression (does NOT support multi-output natively)
    svr = MultiOutputRegressor(SVR(kernel='rbf', gamma='scale'))
    # Creating a pipeline
    pipe = Pipeline(steps=[('pca', pca), 
                           ('svr', svr)])
    # Parameters of pipeline for the randomized search with cross-validation
    param_dists = {'pca__n_components': stats.uniform(),
                   'svr__estimator__C': stats.lognorm(1e0, 1e3),
                   'svr__estimator__epsilon': stats.lognorm(1e-5, 1e-2),
                  }
else:
    raise NotImplementedError('Model name "{}" is not recognized or implemented!'.format(model))

## Model

In [7]:
n_input =4
n_out = 4#96
step=1#向前移动步数
UPS=1###表示多久更新一次
###00前期数据准备
Xx, Yy=DataPreprogress(d1_sale,step)
Date1=d1_sale.timestamp[:]#获得对应时间戳
Date1=np.array(Date1).reshape(len(Date1),1)
Date1Xx, Date1Yy = to_supervised2(Date1, n_input, n_out,step)#准备数据#00  
TrainT=0
TestT=0
ForecastSet3=[]
TestSet3=[]
Datetest_x=[]
Datetest_y=[]
abc=30
MiLth=int(24*60/abc)
n_input=4
##数据拆分
SplitNum=int((d1_sale.shape[0]/int(24*60/abc))*0.7)*int(24*60/abc)-48*2-n_input##天数的百分之0.7,再乘以48,测试集的初始值
print('SplitNum',SplitNum)
SplitNum0=SplitNum#-n_input
########################原始序列SplitNum处，对应y的SplitNum0########################
#print((d1_sale.shape[0]-SplitNum0))
XLT=int(MiLth/n_out)##
print('d1_sale.shape[0]',d1_sale.shape[0])
print('len(Date1Yy),len(Yy)',len(Date1Yy),len(Yy))
for SplitNum0 in range(SplitNum0,d1_sale.shape[0],XLT):     ###96表示1天更新一次 
        
    if SplitNum0>len(Date1Yy):###表示Yy没有数据了
        break
    else:
        hhhhh=(SplitNum0+XLT)
        print('1SplitNum0,hhhhh',SplitNum0,hhhhh)
        T0=0
        train_x, test_x = Xx[SplitNum0-10*XLT:SplitNum0], Xx[SplitNum0:hhhhh]
        train_y, test_y = Yy[SplitNum0-10*XLT:SplitNum0], Yy[SplitNum0:hhhhh]
        #print('Yy',Yy)
        Date1test_x,Date1test_y = Date1Xx[SplitNum0:SplitNum0+XLT], Date1Yy[SplitNum0:hhhhh]#Date1Xx, Date1Yy
        #print('Date1test_x',Date1test_x)
        print('Date1test_y[0]',Date1test_y[0])
        #创建模型
        print('SplitNum0-10*XLT,SplitNum0,SplitNum0+XLT',SplitNum0-14*XLT,SplitNum0,SplitNum0+XLT)
        NITER =5 #100  # number of random search iterations
        search_multi = RandomizedSearchCV(estimator=pipe, param_distributions=param_dists, 
                                          cv=TimeSeriesSplit(n_splits=3),
                                          scoring='neg_mean_squared_error',
                                          n_iter=NITER, refit=True, n_jobs=-1)
        search_multi.fit(train_x, train_y)
        TrainT+=time.clock()-T0
        #print('Datetest_x',Datetest_x)
        #print('Datetest_y',Datetest_y)
        T1=time.clock()
        y_predict = search_multi.predict(test_x)#.flatten()
        #for j in range(len(y_predict)):
        for j in range(len(test_y)):
            ForecastSet3.append(y_predict[j])#*(MaxValue-MinValue)+MinValue)
            TestSet3.append(test_y[j])#*(MaxValue-MinValue)+MinValue)
            Datetest_y.append(Date1test_y[j])
        for l in range(len(Date1test_x)):
            Datetest_x.append(Date1test_x[l])
        TestT+=time.clock()-T1    
        print('Best parameter (CV score = {:.3f}):'.format(search_multi.best_score_))
        print(search_multi.best_params_)
print('TrainT,TestT',TrainT,TestT)
#print('TestSet3, ForecastSet3',TestSet3, ForecastSet3)
#AllRMSE,scores,AllMSE,AllMAE,AllMAPE,AllNum=evaluate_forecasts1(np.array(TestSet3), np.array(ForecastSet3))

SplitNum 6956
d1_sale.shape[0] 10080
len(Date1Yy),len(Yy) 10073 10073
1SplitNum0,hhhhh 6956 6968
Date1test_y[0] ['2020-10-28T00:30:00.000000000' '2020-10-28T01:00:00.000000000'
 '2020-10-28T01:30:00.000000000' '2020-10-28T02:00:00.000000000']
SplitNum0-10*XLT,SplitNum0,SplitNum0+XLT 6788 6956 6968


RandomizedSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
                   estimator=Pipeline(steps=[('preprocess', 'passthrough'),
                                             ('svr',
                                              MultiOutputRegressor(estimator=SVR()))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'preprocess': [None, StandardScaler()],
                                        'svr__estimator__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000254E8795470>,
                                        'svr__estimator__epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000254E8795668>},
                   scoring='neg_mean_squared_error')

Best parameter (CV score = -100798.347):
{'preprocess': StandardScaler(), 'svr__estimator__C': 1000.6956264884774, 'svr__estimator__epsilon': 1.0099851376543592}
1SplitNum0,hhhhh 6968 6980
Date1test_y[0] ['2020-10-28T06:30:00.000000000' '2020-10-28T07:00:00.000000000'
 '2020-10-28T07:30:00.000000000' '2020-10-28T08:00:00.000000000']
SplitNum0-10*XLT,SplitNum0,SplitNum0+XLT 6800 6968 6980


RandomizedSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
                   estimator=Pipeline(steps=[('preprocess', 'passthrough'),
                                             ('svr',
                                              MultiOutputRegressor(estimator=SVR()))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'preprocess': [None, StandardScaler()],
                                        'svr__estimator__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000254E8795470>,
                                        'svr__estimator__epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000254E8795668>},
                   scoring='neg_mean_squared_error')

Best parameter (CV score = -88869.068):
{'preprocess': None, 'svr__estimator__C': 1000.639694569467, 'svr__estimator__epsilon': 1.0099991589864854}
1SplitNum0,hhhhh 6980 6992
Date1test_y[0] ['2020-10-28T12:30:00.000000000' '2020-10-28T13:00:00.000000000'
 '2020-10-28T13:30:00.000000000' '2020-10-28T14:00:00.000000000']
SplitNum0-10*XLT,SplitNum0,SplitNum0+XLT 6812 6980 6992


...

Best parameter (CV score = -67838.588):
{'preprocess': None, 'svr__estimator__C': 1000.2889287905506, 'svr__estimator__epsilon': 1.0099913808267047}
1SplitNum0,hhhhh 10052 10064
Date1test_y[0] ['2020-12-31T12:30:00.000000000' '2020-12-31T13:00:00.000000000'
 '2020-12-31T13:30:00.000000000' '2020-12-31T14:00:00.000000000']
SplitNum0-10*XLT,SplitNum0,SplitNum0+XLT 9884 10052 10064


RandomizedSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
                   estimator=Pipeline(steps=[('preprocess', 'passthrough'),
                                             ('svr',
                                              MultiOutputRegressor(estimator=SVR()))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'preprocess': [None, StandardScaler()],
                                        'svr__estimator__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000254E8795470>,
                                        'svr__estimator__epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000254E8795668>},
                   scoring='neg_mean_squared_error')

Best parameter (CV score = -74751.140):
{'preprocess': None, 'svr__estimator__C': 1000.4335811030702, 'svr__estimator__epsilon': 1.0099973527305137}
1SplitNum0,hhhhh 10064 10076
Date1test_y[0] ['2020-12-31T18:30:00.000000000' '2020-12-31T19:00:00.000000000'
 '2020-12-31T19:30:00.000000000' '2020-12-31T20:00:00.000000000']
SplitNum0-10*XLT,SplitNum0,SplitNum0+XLT 9896 10064 10076


RandomizedSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
                   estimator=Pipeline(steps=[('preprocess', 'passthrough'),
                                             ('svr',
                                              MultiOutputRegressor(estimator=SVR()))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'preprocess': [None, StandardScaler()],
                                        'svr__estimator__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000254E8795470>,
                                        'svr__estimator__epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000254E8795668>},
                   scoring='neg_mean_squared_error')

Best parameter (CV score = -78994.170):
{'preprocess': StandardScaler(), 'svr__estimator__C': 1009.0421644956355, 'svr__estimator__epsilon': 1.009987895257041}
TrainT,TestT 2972.6469621 0.38149620000002293


# Forecasting Test

## Define functions

In [8]:
#03评估预测误差
def evaluate_forecasts1(actual, predicted):#返回预测值的总RMSE以及每天RMSE的列表
    scores = list() 
    for i in range(actual.shape[1]):
        mse = mean_squared_error(actual[:, i], predicted[:, i])#计算MSE
        rmse = sqrt(mse)#计算RMSE
        scores.append(rmse)#计算每个预测值的RMSE,存放在列表#这个现在来看没有多大意义，因为不是同一个时刻,表示向前1步、2步、几步的预测效果
    s = 0
    sMAE = 0
    sMAPE = 0
    RMSE1=list()
    for row in range(actual.shape[0]):
        ss=0
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col]) ** 2
            ss+= (actual[row, col] - predicted[row, col]) ** 2
            sMAE+= np.abs(actual[row, col] - predicted[row, col])
            sMAPE+= np.abs((actual[row, col] - predicted[row, col])/actual[row, col])
        RMSE1.append(sqrt(ss/actual.shape[1]))##每一组的RMSE
    #print('RMSE1',RMSE1)
    AllNum=actual.shape[0] * actual.shape[1]
    AllRMSE = sqrt(s / AllNum)#计算所有值的RMSE
    AllMSE = s / AllNum#计算所有值的RMSE
    AllMAE =sMAE/AllNum
    AllMAPE =sMAPE/AllNum  
    name='[Num]/RMSE/MSE/MAE/MAPE'
    SingPredictscores = list() 
    SingPredictscores.append(AllRMSE)
    SingPredictscores.append(AllMSE)
    SingPredictscores.append(AllMAE)
    SingPredictscores.append(AllMAPE)
    SingPredictscores = ', '.join(['%.1f' % s for s in SingPredictscores])
    print('AllNum,AllRMSE,AllMSE,AllMAPE,AllMAE',AllNum,AllRMSE,AllMSE,AllMAPE,AllMAE)
    #print('%s: [%d] %s' % (name,AllNum,SingPredictscores))
    return AllRMSE,scores,AllMSE,AllMAE,AllMAPE,AllNum

###04 测试部分，预测结果的传统评价
def  ValueTestArima(test_y,predictionspredList):
    ###得到测试y值的原始值
    test_y1=test_y.copy()## 对原始的array1的复制
    for i in range(test_y1.shape[0]):
        test_y1[i, :]=test_y1[i, :]#*(dfghu.iloc[dfghu.shape[0]-1,2]-dfghu.iloc[0,2])+dfghu.iloc[0,2]
        #*(MaxValue-MinValue)+MinValue#不用求exp啦
    PredictScresSet=list()
    for preddd in predictionspredList:
        for predictionspredI in preddd:
            PredictScresSet.append([])
            #score, scores = evaluate_forecasts1(test_y1[:, :], predictionspredI)
            AllRMSE,scores,AllMSE,AllMAE,AllMAPE,AllNum=evaluate_forecasts1(test_y1[:, :], predictionspredI)
            PredictScresSet[-1].append(AllRMSE)
            PredictScresSet[-1].append(AllMSE)
            PredictScresSet[-1].append(AllMAE)
            PredictScresSet[-1].append(AllMAPE)
            PredictScresSet[-1].append(AllNum)
            #print('所有值的RMSE',score)
            #summarize_scores('lstm', AllRMSE, scores)#总结分数
            days = range(n_out) 
    return PredictScresSet

## Evaluation Metrics

In [9]:
yhat_sequence1=TestSet3
NewForecastSet11=ForecastSet3#每天更新参数
#print('ForecastSet3',ForecastSet3)
#NewForecastSet11=y_predict[:len(yhat_sequence1)]#不更新参数
#print('00NewForecastSet11',NewForecastSet11)#测试值
###预测值格式化
predictionspredList=[]#得到多个分位数预测模型、不同重复次数下的预测值集合（predictionspredList）
predictionspredList.append([])
predictionspredList[-1].append([])
for ij in range(len(NewForecastSet11)):
    for h in range(len(NewForecastSet11[ij])):
        NewForecastSet11[ij][h]=NewForecastSet11[ij][h]#*(dfghu.iloc[dfghu.shape[0]-1,2]-dfghu.iloc[0,2])+dfghu.iloc[0,2]
        #*(MaxValue-MinValue)+MinValue
    #print('原值',NewForecastSet1[ij]*(MaxValue-MinValue)+MinValue)
    predictionspredList[-1][-1].append(NewForecastSet11[ij])               
predictionspredList[-1][-1] = array(predictionspredList[-1][-1])#list转换为数组
#print('predictionspredList',predictionspredList)
###04 测试部分，预测结果的传统评价
ArimaPredictScresSet0=ValueTestArima(np.array(yhat_sequence1),np.array(predictionspredList))
################准确度评价指标################（这是仅仅基于一个个的预测值的）
#print('ArimaPredictScresSet0',ArimaPredictScresSet0)
ForecastResult = pd.DataFrame(columns = ['Name','RMSE','MAE']) 
ForecastResult0 = pd.DataFrame(columns = ['Name','RMSE','MAE'])
ForecastResult0.loc[0,'Name']='Arima Model'
ForecastResult0.loc[0,'RMSE']=round(ArimaPredictScresSet0[0][0],2)
ForecastResult0.loc[0,'MAE']=round(ArimaPredictScresSet0[0][2],2)

ForecastResult=ForecastResult0
display(ForecastResult.tail(8))

AllNum,AllRMSE,AllMSE,AllMAPE,AllMAE 12468 324.6171419567706 105376.28885218213 inf 235.07394345279195


D:\anaconda\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars


,Name,RMSE,MAE
0,Arima Model,324.62,235.07


#  Determine total cost and service level

In [10]:
#先把真实的数据中的test之后的搞过来
ReStartStock=60000#44128#34475#13790#该变量表示补货到达后的初始库存，1吨92#汽油=1379L
IntervalT=n_input/2
df_test1  = d1_sale[(pd.to_datetime(d1_sale.timestamp)>=pd.to_datetime(Datetest_x[0][0]))]#df是按照单位时间（abc）聚合后的所有数据
df_test1.index = pd.DatetimeIndex(df_test1['timestamp'])
print('df_test1',df_test1)
TestDFStockOut1=DFStockOut[(DFStockOut.timestamp>=pd.to_datetime(Datetest_y[0][0]))]
TestDFStockOut1.index = pd.DatetimeIndex(TestDFStockOut1['timestamp'])
print('TestDFStockOut1',TestDFStockOut1)

df_test1                               timestamp   value        Date  MinuteLabel
timestamp                                                               
2020-10-27 22:30:00 2020-10-27 22:30:00  507.54  2020-10-27           44
2020-10-27 23:00:00 2020-10-27 23:00:00  307.62  2020-10-27           45
2020-10-27 23:30:00 2020-10-27 23:30:00  280.35  2020-10-27           46
2020-10-27 23:59:00 2020-10-27 23:59:00  179.64  2020-10-27           47
2020-10-28 00:30:00 2020-10-28 00:30:00   38.89  2020-10-28            0
...                                 ...     ...         ...          ...
2020-12-31 22:00:00 2020-12-31 22:00:00  561.08  2020-12-31           43
2020-12-31 22:30:00 2020-12-31 22:30:00   94.09  2020-12-31           44
2020-12-31 23:00:00 2020-12-31 23:00:00   80.46  2020-12-31           45
2020-12-31 23:30:00 2020-12-31 23:30:00  288.83  2020-12-31           46
2020-12-31 23:59:00 2020-12-31 23:59:00   76.19  2020-12-31           47

[3124 rows x 4 columns]
TestDFStockOut1  

In [11]:
###预测值减100
predictionspredList11=[]
for i in range(len(predictionspredList)):
    predictionspredList11.append([])
    for j in range(len(predictionspredList[i])):
        predictionspredList11[-1].append([])
        for k in range(len(predictionspredList[i][j])):
            predictionspredList11[-1][-1].append([])
            #print('predictionspredList[i][j][k]',predictionspredList[i][j][k])
            for ij in range(len(predictionspredList[i][j][k])):
                predictionspredList11[-1][-1][-1].append(0)
                predictionspredList11[i][j][k][ij]=predictionspredList[i][j][k][ij]#-100#前面减了100，得减一下
                #predictionspredList11[i][j][k][ij]=predictionspredList[i][j][k][ij]-100#销量的预测值"""

df_testXX=df_test1.iloc[:,0:3]#Date列、以及UnitSales列
#df_testXX['value']=df_testXX['value'].apply(lambda x: x-100)#减去加的那100
df_testXX['value']=df_testXX['value'].apply(lambda x: x)#减去加的那100
df_testXX['timestamp'] = pd.to_datetime(df_testXX['timestamp'])#为什么成了选取偶数
df_testXX['Hour'] = df_testXX['timestamp'].dt.hour.astype("int") #转化提取分钟
df_testXX['Minute'] = df_testXX['timestamp'].dt.minute.astype("int") #转化提取分钟
##固定参数，取值不会发生变化
ReStartStock=60000#44128#34475#13790#该变量表示补货到达后的初始库存，1吨92#汽油=1379L
IntervalT=1.5#表示补货到达所需时间

c1=1#c1表示消耗量所在列
a1=df_testXX.shape[1]
df_testXX['OurLeaveStock']=df_testXX['timestamp'].apply(lambda x : ReStartStock)#5
b1=a1+1
df_testXX['OurBuHuoStart']=df_testXX['timestamp'].apply(lambda x : 0)#补货触发时刻7
#display(df_testXX.head(2))#按照单位时间聚合

In [12]:
###参数更新
def updateCanShu(DHCS1,df_testXX,mm,IntervalT,ReStartStock,a1,NewTC0,NoOS):#更新参数的自定义函数
    NoOS+=1#
    ActualTimeOS=0
    PredictTimeOS=df_testXX.iloc[mm+3,0]#预测的缺货时刻
    ActualDate=0
    DHCS1=DHCS1+1
    #tt=df_testXX.iloc[mm,3]+df_testXX.iloc[mm,4]/60+IntervalT##tt表示补货到达时刻
    if df_testXX.iloc[mm,3]+df_testXX.iloc[mm,4]/60==23+(59/60):       
        tt=0+IntervalT##tt表示补货到达时刻
        ###111###print('00中tt',tt)
    else:
        tt= df_testXX.iloc[mm,3]+df_testXX.iloc[mm,4]/60+IntervalT##tt表示补货到达时刻
        ###111###print('11中tt',tt)
    dd=1
    BHVolume=ReStartStock-df_testXX.iloc[mm,a1]#补货量10000-当前库存水平
    NewTC0=NewTC0+Dcc0#订购一次1500元
    return DHCS1,tt,dd,BHVolume,NewTC0,NoOS,PredictTimeOS,ActualTimeOS, ActualDate
###变动的参数
hh=1100/(365*24*1378)# 1吨≈1378L，持有成本1100元/吨/年
tt=0#表示补货时间
DHCS1=0#表示订货次数
NoOfStockout0=0
dd=0
###计算库存成本相关的参数
tt1=df_testXX.iloc[0,0]#表示上一次补货到达时刻
ReStartStock00=ReStartStock##初始化周期初，库存水平
NewTC0=0
global Dcc0
Dcc0=150#一次订购成本
##生成缺货情况表
dfTimeOutOfStock= pd.DataFrame(columns=('No', 'PredictTime','ActualDate', 'ActualTime','InventoryLevel'))
NoOS=-1#
InventoryLevel=0#补货到达时，库存水平
PredictTimeOS=0#预测的缺货时刻
ActualDate=0
ActualTimeOS=0#实际缺货时刻
####第一组,重新计算剩余库存以初始化
for i in range(len(Datetest_x[0])):
    if i==0:
        df_testXX.iloc[i,a1]=df_testXX.iloc[i,a1]-df_testXX.iloc[i,c1]
    else:             
        df_testXX.iloc[i,a1]=df_testXX.iloc[i-1,a1]-df_testXX.iloc[i,c1]#表示剩余库存水平#
###111###print('df_testXX.iloc[mm,a1]',df_testXX.iloc[0,a1])
mm=i#初始化mm.注，预测的是Datetest_y，其对应的第一组值为 [['2017-03-11T01:30:00.000000000' '2017-03-11T02:00:00.000000000' '2017-03-11T02:30:00.000000000']
###我的预测方法下预测的缺货时间
###111###print('我的预测方法下预测的缺货时间')
for k in range(1,len(predictionspredList[-1][0]),1): ##除了第一组，后面只取预测的第一个值

##00未来需求的求和，对应step3                
    SumDemand=0##SumDemand表示未来1个时段内的总需求
    for h in range(len(predictionspredList11[-1][0][k])):
        SumDemand=SumDemand+predictionspredList11[-1][0][k][h]       
    SumDemandPus1=SumDemand#SumDemandPus1表示未来2个时段内的总需求
    if k==len(predictionspredList[-1][0])-1:
        SumDemandPus1= SumDemandPus1+0#如果是最后一个则+0
    else:
        #print('predictionspredList11[-1][0]',len(predictionspredList11[-1][0]))
        #print('predictionspredList[-1][0]',len(predictionspredList[-1][0]))
        SumDemandPus1= SumDemandPus1+predictionspredList11[-1][0][k+1][-1]#只需向前多滚动一个单位时间即可
###01对应缺货时间判断        
    if SumDemand>=df_testXX.iloc[mm,a1] and tt==0:#tt=0表示首次补货
         ###111###print('\033[0;36m前期没做好，来不及实施应急补货，\033[0m')
         ###111###display(df_testXX.iloc[mm,0])
         DHCS1,tt,dd,BHVolume,NewTC0,NoOS,PredictTimeOS,ActualTimeOS,ActualDate=updateCanShu(DHCS1,df_testXX,mm,IntervalT,ReStartStock,a1,NewTC0,NoOS)
    else:
            if SumDemandPus1>df_testXX.iloc[mm,a1] and tt==0:#（2）去掉了=
                     ###111###print('\033[0;32m不能满足SumDemandPus1导致的缺货时间00为\033[0m',df_testXX.iloc[mm,0])                 

                     DHCS1,tt,dd,BHVolume,NewTC0,NoOS,PredictTimeOS,ActualTimeOS,ActualDate=updateCanShu(DHCS1,df_testXX,mm,IntervalT,ReStartStock,a1,NewTC0,NoOS)
            else:

                TestDFStockOut2 = DFStockOut[(DFStockOut.timestamp==pd.to_datetime(df_testXX.iloc[mm+1,0]))]#mm确实应该加1###TestDFStockOut2表示未来第一个时段内的所有真实需求
                ###TestDFStockOut2表示未来第一个时段内的所有真实需求
                TestDFStockOut2 = TestDFStockOut2.sort_values(by=['timestamp'],ascending=[True])#排序
                singleSumDemand=0
                for ii in range(TestDFStockOut2.shape[0]):
                    singleSumDemand=singleSumDemand+TestDFStockOut2.iloc[ii,3]
                    if singleSumDemand>=predictionspredList11[-1][0][k][0]:#未未来第一个时段内的预测需求
                        if df_testXX.iloc[mm,a1]-singleSumDemand <= (SumDemandPus1-singleSumDemand )and tt==0:                            
                        ##现在是按照阶段更新的，如何把实时和时段结合，把该时段的剩余部分忽略
                            ##如果现在的库存水平支撑不了未来（当前库存水平<剩余未来时段预测总量），则此刻抓紧(当前库存水平-已经发生的库存消耗量<=剩下两端的库存消耗量)
                             print('事件驱动下的缺货时间11为',TestDFStockOut2.iloc[ii,0], TestDFStockOut2.iloc[ii,1])
                             NoOS+=1
                             PredictTimeOS=TestDFStockOut2.iloc[ii+3,-1]
                             DHCS1=DHCS1+1
                             NewTC0=NewTC0+Dcc0
                             tt=TestDFStockOut2.iloc[ii,2]+IntervalT#补货到达时间
                             ActualTimeOS=0
                             ActualDate=0
                             dd=1
                             BHVolume=ReStartStock-(df_testXX.iloc[mm,a1]-singleSumDemand)#补货量
                             continue#出现了缺货时间后，这部分就可以不用继续拉                       
##02获得实际缺货的时间（主要是防止真实需求的波动性，可能会导致缺货时间发生在补货到达前）
    if tt!=0 and dd==1:##dd==0 or 1用来调节 缺货时间只有一个，库存水平后面不能继续为负
        #if tt>=24.0:#补货到达时间24和0的问题
            #tt=tt-24-0.0001#对于24、24.5等情况的处理
        TestDFStockOut3 = DFStockOut[(DFStockOut.timestamp==pd.to_datetime(df_testXX.iloc[mm+1,0]))]###TestDFStockOut2表示未来第一个时段内的所有真实需求
        TestDFStockOut3 = TestDFStockOut3.sort_values(by=['timestamp'],ascending=[True])#排序
        singleSumDemand1=0
        for ii in range(TestDFStockOut3.shape[0]):
            singleSumDemand1=singleSumDemand1+TestDFStockOut3.iloc[ii,3]
            if df_testXX.iloc[mm,a1]-singleSumDemand1 <= 0 and dd==1:
                NoOfStockout0=NoOfStockout0+1#该参数表示实际缺货次数
                ###111###print('\033[0;35m实际缺货时间是\033[0m',TestDFStockOut3.iloc[ii,0], TestDFStockOut3.iloc[ii,1])
                ActualTimeOS= TestDFStockOut3.iloc[ii,1]
                ActualDate= TestDFStockOut3.iloc[ii,0]
                ###111###print('此时的库存水平',df_testXX.iloc[mm,a1]-singleSumDemand1) 
                dd=2#用来调节库存最小为0，不能为负

###03 该部分类似于更新，补货到达事件和库存消耗事件  
    mm=mm+1#向前滚动一个时间单位, 应该放到最后
    BJtt = df_testXX.iloc[mm,3]+ df_testXX.iloc[mm,4]/60   #转化为实数版本的小时  
    if tt!=0: ###补货到达 （BJtt为大于tt的最小时间）
        if tt>=24.0:###对于补货到达时间是24.5、25等情况怎么处理
            if BJtt==23+(59/60):
                tt=tt-24
                BJtt=0
                ###111###print('补货到达时刻为',df_testXX.iloc[mm,0], BJtt)##tt才是补货到达时刻
                ###111###print('tt',tt)##tt才是补货到达时刻
        if tt<=BJtt: ###补货到达 （BJtt为大于tt的最小时间）
        #if tt!=0 and tt<=BJtt: ###补货到达 （BJtt为大于tt的最小时间）
            ###111###print('补货到达时刻为',df_testXX.iloc[mm,0], BJtt)
            if dd==2:
                 ###111###print('\033[5;35m之前出现了缺货，但货物在该时间戳内到达\033[0m')#说明前面就出现了缺货,即(df_testXX.iloc[mm-1,a1]-df_testXX.iloc[mm,c1]<0)，而库存水平≥0，所以令其为0
                 df_testXX.iloc[mm-1,a1]=0
                 #df_testXX.iloc[mm,a1]=0+BHVolume
                 df_testXX.iloc[mm,a1]=ReStartStock
                 InventoryLevel=0
                 TimeOfDiff=(((df_testXX.iloc[mm,0]-tt1).days)*24+(((df_testXX.iloc[mm,0]-tt1).seconds)/60)/60)#两次补货之间的时间差
                 CYC00=hh*TimeOfDiff*((ReStartStock00+(df_testXX.iloc[mm-1,a1]))/2)#持有成本
            else:#补货到达后的库存水平变化
                #df_testXX.iloc[mm,a1]=df_testXX.iloc[mm-1,a1]-df_testXX.iloc[mm,c1]+BHVolume
                df_testXX.iloc[mm,a1]=ReStartStock
                InventoryLevel=df_testXX.iloc[mm-1,a1]-df_testXX.iloc[mm,c1]
                TimeOfDiff=(((df_testXX.iloc[mm,0]-tt1).days)*24+(((df_testXX.iloc[mm,0]-tt1).seconds)/60)/60)#两次补货之间的时间差
                CYC00=hh* TimeOfDiff*((ReStartStock00+(df_testXX.iloc[mm-1,a1]-df_testXX.iloc[mm,c1]))/2)#持有成本

            ###111###print('No\PredictTim\\ActualDate\ActualTime\InventoryLevel',NoOS, PredictTimeOS,ActualDate,ActualTimeOS,InventoryLevel)
            new=pd.DataFrame({'No':NoOS, 'PredictTime':PredictTimeOS,'ActualDate':ActualDate,'ActualTime':ActualTimeOS,'InventoryLevel':InventoryLevel},index=[1]) 
            dfTimeOutOfStock=dfTimeOutOfStock.append(new,ignore_index=True) 

            ReStartStock00=df_testXX.iloc[mm,a1]#变量ReStartStock00表示新周期的初始库存
            tt1=df_testXX.iloc[mm,0]#代表上一次补货时+
            NewTC0=NewTC0+CYC00
            tt=0   
            dd=0#用以表示    
        else: ###没发出补货或者补货没有到达
            if dd==2:###库存水平降为0
                ###111###print('\033[1;35m出现了缺货，但货物还没到达\033[0m')
                df_testXX.iloc[mm,a1]=0##说明前面就出现了缺货，系统停止，库存水平为0
                continue#说明实际到了缺货时间，系统停止，下面循环不用走
            else:##事件驱动库存消耗
                df_testXX.iloc[mm,a1]=df_testXX.iloc[mm-1,a1]-df_testXX.iloc[mm,c1]#因为是滚动一个单位向前的，所以要这样 
    else: ###没发出补货或者补货没有到达
            if dd==2:###库存水平降为0
                ###111###print('\033[1;35m出现了缺货，但货物还没到达\033[0m')
                df_testXX.iloc[mm,a1]=0##说明前面就出现了缺货，系统停止，库存水平为0
                continue#说明实际到了缺货时间，系统停止，下面循环不用走
            else:##事件驱动库存消耗
                df_testXX.iloc[mm,a1]=df_testXX.iloc[mm-1,a1]-df_testXX.iloc[mm,c1]#因为是滚动一个单位向前的，所以要这样
    if k==len(predictionspredList[-1][0])-1:
            InventoryLevel=max(df_testXX.iloc[mm-1,a1]-df_testXX.iloc[mm,c1],0)
            TimeOfDiff=(((df_testXX.iloc[mm,0]-tt1).days)*24+(((df_testXX.iloc[mm,0]-tt1).seconds)/60)/60)#两次补货之间的时间差
            CYC00=hh* TimeOfDiff*((ReStartStock00+InventoryLevel)/2)#持有成本
            NewTC0=NewTC0+CYC00
#display(dfTimeOutOfStock.head(10))


In [13]:
dfTimeOutOfStock.to_csv('SVM--92H+0.99OurMethoddfTimeOutOfStock.csv')

In [14]:
ComparResultdf = pd.DataFrame(columns = ['Number of replenishment','Inventory Cost','Delivery Cost','Total Cost','Number of Stock-out']) 
ComparResultdf1 = pd.DataFrame(columns=['name','Number of replenishment','Inventory Cost','Delivery Cost','Total Cost','Number of Stock-out'])
ComparResultdf1.loc[0,'name']='SVM Method'
ComparResultdf1.loc[0,'Number of replenishment']=DHCS1
ComparResultdf1.loc[0,'Inventory Cost']=round(NewTC0,2)
ComparResultdf1.loc[0,'Delivery Cost']=round(DHCS1*(213+1.83*50),2)
ComparResultdf1.loc[0,'Total Cost']=round(NewTC0+DHCS1*(213+1.83*50),2)
ComparResultdf1.loc[0,'Number of Stock-out']=NoOfStockout0
ComparResultdf1.loc[0,'SL']=round(100*((65-NoOfStockout0)/65),2)
ComparResultdf=ComparResultdf1
display(ComparResultdf.head(9))

,name,Number of replenishment,Inventory Cost,Delivery Cost,Total Cost,Number of Stock-out,SL
0,SVM Method,37,9907.1,11266.5,21173.6,4,93.85
